# FILTRADO DE DATOS


In [6]:
# Conectar a la base de datos
import pandas as pd
import sqlite3

conexion = sqlite3.connect('database.sqlite')

# Leer las tablas como DataFrames
df = pd.read_sql_query("SELECT * FROM apicall", conexion)
df2 = pd.read_sql_query("SELECT * FROM commerce", conexion)

# Crear una función para calcular el total a pagar por empresa
def calcular_total_empresa(df, commerce_id, contrato, mes):
    # Filtrar los datos de la empresa
    data_empresa = df[df['commerce_id'] == commerce_id]

    # Filtrar por el mes y el año 2024
    data_empresa['date_api_call'] = pd.to_datetime(data_empresa['date_api_call'])
    data_empresa = data_empresa[(data_empresa['date_api_call'].dt.month == mes) &
                                 (data_empresa['date_api_call'].dt.year == 2024)]

    # Separar peticiones exitosas y no exitosas
    exitosas = data_empresa[data_empresa['ask_status'] == 'Successful']
    no_exitosas = data_empresa[data_empresa['ask_status'] != 'Successful']

    # Contar peticiones
    total_exitosas = len(exitosas)
    total_no_exitosas = len(no_exitosas)

    # Calcular el costo según el contrato
    if contrato == "QuantumLeap":
        costo_base = total_exitosas * 600

    elif contrato == "Zenith":
        if total_exitosas <= 22000:
            costo_base = total_exitosas * 250
        else:
            costo_base = total_exitosas * 130

        # Descuento por más de 6,000 peticiones no exitosas
        if total_no_exitosas > 6000:
            descuento = 0.05 * costo_base
            costo_base -= descuento
            costo_base *= 1.15  # Incrementar un 15% si se aplica el descuento

    elif contrato == "FusionWave":
        costo_base = total_exitosas * 300

        # Descuento por peticiones no exitosas
        if 2500 <= total_no_exitosas <= 4500:
            descuento = 0.05 * costo_base
            costo_base -= descuento
            costo_base *= 1.15  # Incrementar un 15% si se aplica el descuento
        elif total_no_exitosas > 4501:
            descuento = 0.08 * costo_base
            costo_base -= descuento
            costo_base *= 1.15  # Incrementar un 15% si se aplica el descuento

    elif contrato == "NexaTech":
        if total_exitosas <= 10000:
            costo_base = total_exitosas * 250
        elif 10001 <= total_exitosas <= 20000:
            costo_base = total_exitosas * 200
        else:
            costo_base = total_exitosas * 170

    # Calcular IVA
    iva = costo_base * 0.19
    total_pagar = costo_base + iva

    # Retornar resultados
    return {
        "Empresa": contrato,
        "Mes": mes,
        "Exitosas": total_exitosas,
        "No exitosas": total_no_exitosas,
        "Costo base (sin IVA)": costo_base,
        "IVA (19%)": iva,
        "Total a pagar (con IVA)": total_pagar
    }

# Definir los contratos de cada empresa (excluyendo Innovexa Solutions debido a que no encuentra activa)
empresas = {
    "QuantumLeap": "Rh2k-J1o7-zndZ-cOo8",
    "Zenith": "3VYd-4lzT-mTC3-DQN5",
    "FusionWave": "GdEQ-MGb7-LXHa-y6cd",
    "NexaTech": "Vj9W-c4Pm-ja0X-fC1C"
}

# Calcular los resultados por separado para julio y agosto
resultados_julio = []
resultados_agosto = []

for contrato, commerce_id in empresas.items():
    resultado_julio = calcular_total_empresa(df, commerce_id, contrato, 7)
    resultado_agosto = calcular_total_empresa(df, commerce_id, contrato, 8)
    resultados_julio.append(resultado_julio)
    resultados_agosto.append(resultado_agosto)

# Convertir los resultados en DataFrames
df_resultados_julio = pd.DataFrame(resultados_julio)
df_resultados_agosto = pd.DataFrame(resultados_agosto)

# Mostrar los resultados
print("Resultados de julio:")
print(df_resultados_julio)

print("\nResultados de agosto:")
print(df_resultados_agosto)


<ipython-input-6-f14311e14909>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_empresa['date_api_call'] = pd.to_datetime(data_empresa['date_api_call'])
<ipython-input-6-f14311e14909>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_empresa['date_api_call'] = pd.to_datetime(data_empresa['date_api_call'])
<ipython-input-6-f14311e14909>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

Resultados de julio:
       Empresa  Mes  Exitosas  No exitosas  Costo base (sin IVA)    IVA (19%)  \
0  QuantumLeap    7     20231         5234            12138600.0  2306334.000   
1       Zenith    7     20394         5046             5098500.0   968715.000   
2   FusionWave    7     20406         5115             6476864.4  1230604.236   
3     NexaTech    7     20387         5029             3465790.0   658500.100   

   Total a pagar (con IVA)  
0             1.444493e+07  
1             6.067215e+06  
2             7.707469e+06  
3             4.124290e+06  

Resultados de agosto:
       Empresa  Mes  Exitosas  No exitosas  Costo base (sin IVA)    IVA (19%)  \
0  QuantumLeap    8     20304         5066            12182400.0  2314656.000   
1       Zenith    8     20414         5038             5103500.0   969665.000   
2   FusionWave    8     20262         5117             6431158.8  1221920.172   
3     NexaTech    8     20358         5030             3460860.0   657563.400   


<ipython-input-6-f14311e14909>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_empresa['date_api_call'] = pd.to_datetime(data_empresa['date_api_call'])


# CREACION DE LA TABLA RESUMEN

In [7]:
from datetime import datetime

# Añade las columnas adicionales necesarias (Nombre, Nit, y Correo)
datos_empresas = {
    "Innovexa": {"Nombre": "Innovexa Solutions", "Nit": "445470636", "Correo": "innovexasolutions@microfitsof.com"},
    "QuantumLeap": {"Nombre": "QuantumLeap Inc", "Nit": "198818316", "Correo": "quantumleapinc.@gemaily.net"},
    "Zenith": {"Nombre": "Zenith Corp", "Nit": "28960112", "Correo": "zenithcorp.@gemaily.net"},
    "FusionWave": {"Nombre": "FusionWave Enterprises", "Nit": "919341007", "Correo": "fusionwaveenterprises@microfitsof.com"},
    "NexaTech": {"Nombre": "NexaTech Industries", "Nit": "452680670", "Correo": "nexatechindustries@gemaily.net"}
}

# Crear una lista de filas con los datos finales
resumen_julio = []
for index, row in df_resultados_julio.iterrows():
    empresa = row["Empresa"]
    fecha_mes = "2024-07"
    nombre = datos_empresas[empresa]["Nombre"]
    nit = datos_empresas[empresa]["Nit"]
    correo = datos_empresas[empresa]["Correo"]
    valor_comision = row["Costo base (sin IVA)"]
    valor_iva = row["IVA (19%)"]
    valor_total = row["Total a pagar (con IVA)"]

    # Crear el diccionario de la fila y añadirlo al resumen
    resumen_julio.append({
        "Fecha-Mes": fecha_mes,
        "Nombre": nombre,
        "Nit": nit,
        "Valor_comision": valor_comision,
        "Valor_iva": valor_iva,
        "Valor_Total": valor_total,
        "Correo": correo
    })

# Convertir el resumen en un DataFrame
df_resumen_julio = pd.DataFrame(resumen_julio)

# Formatear columnas para mostrar valores numéricos correctamente
columns_to_format = ["Valor_comision", "Valor_iva", "Valor_Total"]
df_resumen_julio[columns_to_format] = df_resumen_julio[columns_to_format].applymap(lambda x: f'{x:,.2f}')

print(df_resumen_julio)

# Crear una lista de filas con los datos finales
df_resumen_agosto = []
for index, row in df_resultados_agosto.iterrows():
    empresa = row["Empresa"]
    fecha_mes = "2024-08"
    nombre = datos_empresas[empresa]["Nombre"]
    nit = datos_empresas[empresa]["Nit"]
    correo = datos_empresas[empresa]["Correo"]
    valor_comision = row["Costo base (sin IVA)"]
    valor_iva = row["IVA (19%)"]
    valor_total = row["Total a pagar (con IVA)"]

    # Crear el diccionario de la fila y añadirlo al resumen
    df_resumen_agosto.append({
        "Fecha-Mes": fecha_mes,
        "Nombre": nombre,
        "Nit": nit,
        "Valor_comision": valor_comision,
        "Valor_iva": valor_iva,
        "Valor_Total": valor_total,
        "Correo": correo
    })

# Convertir el resumen en un DataFrame
df_resumen_agosto = pd.DataFrame(df_resumen_agosto)

# Formatear columnas para mostrar valores numéricos correctamente
columns_to_format = ["Valor_comision", "Valor_iva", "Valor_Total"]
df_resumen_agosto[columns_to_format] = df_resumen_agosto[columns_to_format].applymap(lambda x: f'{x:,.2f}')

print(df_resumen_agosto)

  Fecha-Mes                  Nombre        Nit Valor_comision     Valor_iva  \
0   2024-07         QuantumLeap Inc  198818316  12,138,600.00  2,306,334.00   
1   2024-07             Zenith Corp   28960112   5,098,500.00    968,715.00   
2   2024-07  FusionWave Enterprises  919341007   6,476,864.40  1,230,604.24   
3   2024-07     NexaTech Industries  452680670   3,465,790.00    658,500.10   

     Valor_Total                                 Correo  
0  14,444,934.00            quantumleapinc.@gemaily.net  
1   6,067,215.00                zenithcorp.@gemaily.net  
2   7,707,468.64  fusionwaveenterprises@microfitsof.com  
3   4,124,290.10         nexatechindustries@gemaily.net  
  Fecha-Mes                  Nombre        Nit Valor_comision     Valor_iva  \
0   2024-08         QuantumLeap Inc  198818316  12,182,400.00  2,314,656.00   
1   2024-08             Zenith Corp   28960112   5,103,500.00    969,665.00   
2   2024-08  FusionWave Enterprises  919341007   6,431,158.80  1,221,920.17  

<ipython-input-7-55f156670ed7>:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_resumen_julio[columns_to_format] = df_resumen_julio[columns_to_format].applymap(lambda x: f'{x:,.2f}')
<ipython-input-7-55f156670ed7>:72: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_resumen_agosto[columns_to_format] = df_resumen_agosto[columns_to_format].applymap(lambda x: f'{x:,.2f}')


In [8]:
df_resumen_total = pd.concat([df_resumen_julio, df_resumen_agosto], axis=0)

df_resumen_total

,Fecha-Mes,Nombre,Nit,Valor_comision,Valor_iva,Valor_Total,Correo
0,2024-07,QuantumLeap Inc,198818316,"12,138,600.00","2,306,334.00","14,444,934.00",quantumleapinc.@gemaily.net
1,2024-07,Zenith Corp,28960112,"5,098,500.00","968,715.00","6,067,215.00",zenithcorp.@gemaily.net
2,2024-07,FusionWave Enterprises,919341007,"6,476,864.40","1,230,604.24","7,707,468.64",fusionwaveenterprises@microfitsof.com
3,2024-07,NexaTech Industries,452680670,"3,465,790.00","658,500.10","4,124,290.10",nexatechindustries@gemaily.net
0,2024-08,QuantumLeap Inc,198818316,"12,182,400.00","2,314,656.00","14,497,056.00",quantumleapinc.@gemaily.net
1,2024-08,Zenith Corp,28960112,"5,103,500.00","969,665.00","6,073,165.00",zenithcorp.@gemaily.net
2,2024-08,FusionWave Enterprises,919341007,"6,431,158.80","1,221,920.17","7,653,078.97",fusionwaveenterprises@microfitsof.com
3,2024-08,NexaTech Industries,452680670,"3,460,860.00","657,563.40","4,118,423.40",nexatechindustries@gemaily.net


# GUARDADO EN LA CARPETA RESULTADOS

In [ ]:
# Guardar los resultados como archivo Excel sin índices en la carpeta de resultados
ruta_excel = "./resultados/df_resultados.xlsx"
df_resumen_total.to_excel(ruta_excel, index=False)
print(f"Copia de seguridad guardada en: {ruta_excel}")

In [9]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders




ENVIAR CORREO ELECTRONICO AL EJECUTOR CON LA TABLA RESUMEN

In [11]:
# Función simple para enviar correo
def enviar_correo_simple(remitente, contraseña, destinatario, asunto, cuerpo, archivo_adjunto):
    try:
        # Configurar el mensaje
        mensaje = MIMEMultipart()
        mensaje["From"] = remitente
        mensaje["To"] = destinatario
        mensaje["Subject"] = asunto
        mensaje.attach(MIMEText(cuerpo, "plain"))

        # Adjuntar archivo
        with open(archivo_adjunto, "rb") as adjunto:
            parte = MIMEBase("application", "octet-stream")
            parte.set_payload(adjunto.read())
        encoders.encode_base64(parte)
        parte.add_header("Content-Disposition", f"attachment; filename={archivo_adjunto}")
        mensaje.attach(parte)

        # Configurar servidor SMTP (Gmail)
        servidor = smtplib.SMTP("smtp.gmail.com", 587)
        servidor.starttls()
        servidor.login(remitente, contraseña)
        servidor.sendmail(remitente, destinatario, mensaje.as_string())
        servidor.quit()

        print(f"Correo enviado a {destinatario}")
    except Exception as e:
        print(f"Error al enviar correo: {e}")

archivo_excel1 = "df_resumen.xlsx"
df_resumen_total.to_excel(archivo_excel1, index=False)

# Configurar datos de correo
remitente = "michellmariana98.12@gmail.com"
contraseña = "edgf jtmr mall zkqn"  # Contraseña de aplicación generada
destinatario = "michellmariana98.12@gmail.com"
asunto = "Resumen en Excel"
cuerpo = "Hola, adjunto el archivo Excel con el resumen solicitado."

# Enviar correo
enviar_correo_simple(remitente, contraseña, destinatario, asunto, cuerpo, archivo_excel1)


Correo enviado a michellmariana98.12@gmail.com
